```python
###### 2.2 Graphs
###### 2.2.1 기본적인 그래픽 함수 이용

import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt

ds = pd.read_csv('buy.csv', header=0, sep=",")

# 산점도
plt.scatter(ds.income, ds.exp)

# 자주 쓰는 함수
plt.hist(ds.income)

x = np.arange(len(ds.income))
plt.bar(x, ds.buy) 
# 원하는 형태 아님
# 범주형 형태로서 바꿀 필요가 있음
x = ds['buy'].value_counts()
plt.bar(x.index, x.values)
# 또는 아래와 같이 가능
x = pd.crosstab(index=ds['buy'], columns='count')
plt.bar(x.index, x['count'])


plt.pie(ds.co)		
# 연속형 변수
# 너무 난잡함. 아래와 같이 변형해보자

x = pd.crosstab(index=pd.cut(ds.co, bins=5), columns='count')
labels = ['A', 'B', 'C', 'D', 'E']
plt.pie(x['count'], labels=labels, autopct='%.1f%%')



from statsmodels.graphics.mosaicplot import mosaic
ds['year'].astype('category') # R에서는 as.factor로 변경 가능!
ds['age'].astype('category')
mosaic(ds, ['year', 'age'], gap=0.01)
# gap : 각 막대그래프에서 위아래 간격을 의미

import seaborn as sns
# 산점도로 이루어진 행렬
sns.pairplot(ds.iloc[:, 5:9], diag_kind='hist')
# 대각행렬은 histogram으로

ds['year'].astype('category')
sns.pairplot(ds, diag_kind='hist', hue='year', vars=['income', 'exp', 'co', 'ad', 'prod'])   
# vars : x축, y축 의미

# 2개의 범주형 변수를 이용한 막대그래프
# 예: 교육수준별 연령대의 분포
bartable = pd.crosstab(ds.edu, ds.age)  
## R에서는 table
bartable.plot.bar(stacked=True, rot=0) # crosstab에서 바로 접근해서 그림 그릴 수 있음
bartable.plot.bar(stacked=False, rot=0)
# rot는 x축 label기울기




####################################################
# 과제2.7
bartable = pd.crosstab(ds.age, ds.year)  ## get the cross tab
bartable.plot.bar(stacked=True, rot=0)
bartable.plot.bar(stacked=False, rot=0)



####################################################


#### 2.2.2 그래프 옵션(options)
# 가장 기본적인 함수: plot(x, y)
plt.scatter(ds.income, ds.exp)
plt.title("Scatter plot")
plt.xlabel("Income")
plt.ylabel("Expense")
plt.xlim(0, 40000)
plt.ylim(0, 30000)

# marker, marker size, color
plt.scatter(ds.income, ds.exp, alpha=0.2, 
            marker = '.', # marker
            s=200*ds.edu, # marker size
            c=ds.age) # marker color
plt.title("Scatter plot")
plt.xlabel("Income")
plt.ylabel("Expense")
plt.xlim(0, 40000)
plt.ylim(0, 30000)


####################################################
# 과제2.8

import seaborn as sns


sns.scatterplot(data=ds, x='income', y='exp', hue='year')
sns.scatterplot(data=ds, x='income', y='exp', hue='year', style='year')

# plt에도 scatter가 있고
# sns에도 scatterplot이 있다!

####################################################


### 다중 플롯
x = np.linspace(0, 2 * np.pi, 400) # 400개로 쪼갬
y = np.sin(x ** 2)

# plt.subplot! 제발 외우자
plt.subplot(2,2,1) # 2행 2열 中 첫 번째 칸에!
plt.plot(x, y)
plt.subplot(2,2,2) # 2행 2열 中 두 번째 칸에!
plt.plot(x, y**2)
plt.subplot(2,2,3) # 2행 2열 中 세 번째 칸에!
plt.plot(x, -y)
plt.subplot(2,2,4) # 2행 2열 中 네 번째 칸에!
plt.plot(x, -y**2)

### 그래프에 text 및 화살표 추가
groups = ds.groupby("year")
for name, group in groups:
    plt.plot(group["income"], group["exp"], marker="o", linestyle="", label=name)
# year로 groupby했기 때문에 알아서 hue='year'의 역할을 함.
plt.legend()
plt.annotate('outlier?', xy=(34600, 10000), xytext=(32600, 5000), arrowprops={'color':'red'})


####################################################
# 과제2.9

from statsmodels.formula.api import ols
import matplotlib.patches as patches #다각형을 그리는 메소드

# 최소제곱법 이용한 단순 회귀
model = ols(formula = 'exp ~ income', data = ds).fit()

groups = ds.groupby("year")
for name, group in groups:
    plt.plot(group["income"], group['exp'],
            marker = "o", linestyle = "", label = name)
plt.legend()
plt.annotate('outlier?', xy = (34600, 10000), xytext = (32600, 5000), arrowprops = {'color':'red'})

#예측값을 이용하여 회귀직선을 그림(모델의 fitted된 값과 x값을 이용)
plt.plot(ds['income'], model.fittedvalues, color = 'black')





#다각형의 좌표를 이용하여 다각형을 그림
point = np.array([[13000,16000],
                 [15000,11000],
                 [25000,16000],
                 [28000,23000]])
shp = patches.Polygon(point, fill = None)
plt.gca().add_patch(shp)
####################################################



#### 2.2.3 ggplot() 함수

import pandas as pd

ds = pd.read_csv('buy.csv', header=0, sep=",")

import matplotlib.pyplot as plt
from plotnine import * 

fig = plt.Figure()
( ggplot(ds) + 
     aes(x="income") +
     geom_histogram(bins=10, fill="red") +
     annotate("text", x=32000, y=100, color = "blue", label = "Outliers") +
     geom_segment(aes(x=32300, y=85, xend=32300, yend=10), color='blue', size=1, arrow = arrow()) +
     annotate("text", x = 20000, y=350, colour = "blue", label = "Most of the distribution is concentrated at") + 
     annotate("text", x = 22000, y=320, colour = "blue", label = "the interval () ...")
)


ds = pd.read_csv('buy.csv', header=0, sep=",")

import matplotlib.pyplot as plt
from plotnine import * 

fig = plt.Figure()
ggplot(ds) +\
aes(x="income") +\
geom_histogram(bins=10, fill="red") +\
annotate("text", x=32000, y=100, color = "blue", label = "Outliers") +\
geom_segment(aes(x=32300, y=85, xend=32300, yend=10), color='blue', size=1, arrow = arrow()) +\
annotate("text", x = 20000, y=350, colour = "blue", label = "Most of the distribution is concentrated at") +\
annotate("text", x = 22000, y=320, colour = "blue", label = "the interval () ...")

# 아니면 괄호를 묶어도 되나봄

####################################################
# 과제2.10

import pandas as pd

ds = pd.read_csv('buy.csv', header=0, sep=",")

import matplotlib.pyplot as plt
from plotnine import * 

fig = plt.Figure() ###
( ggplot(ds) + 
     aes(x="exp") +
     geom_histogram(bins=10, fill="red") 
)


(ggplot(ds) + 
     aes(x="income",y="exp") +
     geom_point()
)

####################################################


################################################
###### 2.3 반복문, 조건문, 함수
################################################

### for 문
import pandas as pd
import numpy as np

ds = pd.read_csv('buy.csv', header=0, sep=",")
n = len(ds)
# R에서는 nrow(ds)로 해결

# income 의 합 : 반복문 이용
total = 0
for i in range(0, n) :
	total = total + ds.loc[i, 'income']
    # 여기서 total쓰면 안 됨
    # for문이라 그럼 매번 total값을 반환할 것임
    

# income 의 합 : 함수 이용
total = ds['income'].sum()


### if/else, ifelse 문
n = len(ds)

if (n > 100) :
    print("The number of data > 100")
else : 
    print("The number of data <= 100")


# ('T', 'F') ⇌ (0, 1) 형태로 변환
ds['buy'] = ds['buy'].apply(lambda x: 'F' if x==0 else 'T')
# lambda안에 if조건문 넣을 수 있다!!
#파이썬에서 "lambda" 는 런타임에 생성해서 사용할 수 있는 익명 함수

# 파이썬에서의 np.where = R에서의 ifelse

### for, if 문
ds = pd.read_csv('buy.csv', header=0, sep=",")
n = len(ds)

# buy==1인 경우의 income의 합 계산 (반복문과 함께 이용)
total = 0

for i in range(0, n) :
    if (ds.loc[i, 'buy']==1) : # 이때의 i에 대해서 수행할 것임
        total = total + ds.loc[i, 'income']
        # total은 이 for문 끝나고 다른 줄에 total입력으로 반환해야함

## buy==1인 경우 데이터 1행씩 추가
ds = pd.read_csv('buy.csv', header=0, sep=",")
n = len(ds)

tdata = pd.DataFrame()


for i in range(0, n) :
    if (ds.loc[i, 'buy']==1) :
        tt = ds.loc[[i]] # 행 추출
        tdata = tdata.append(tt)
        # pd.DataFrame에서 데이터 덧붙이려면? append 이용하자!
        

        
### 함수
import pandas as pd
# R에서는
# get_data = function(file_name, item_no){~ 
# 이렇게 갔을 것임



def get_data (file_name, item_no) :
    ds = pd.read_csv(file_name, header=0)
    sub_data = ds[ds.itemno == item_no]
    
    return (sub_data)


f_n = "func_ex1.csv"
i_no = 21
data1 = get_data (f_n, i_no)

f_n = "func_ex2.csv"
i_no = 21
data2 = get_data(f_n, i_no)

# 데이터 합치기
data = pd.concat([data1, data2]) # list로 넣어준다.
# 넘파이에선 합칠때
# np.concatenate 썼던 것 같음


####################################################
# 과제2.11
import pandas as pd
import numpy as np

ds = pd.read_csv('buy.csv', header=0, sep=",")
ds2 = ds[ds.buy == 0]
n2 = len(ds2)

# income 의 합 : 반복문 이용
# buy가 0일때만 합???
# 이때 pandas의 series를 tolist()로 바꿔서 함수안에서 이용하는 방식 기억해두자
# google에 series를 list로 바꾸는 방법 검색해도 될 듯
def aver (v) :
    
    sum1 = 0
    for i in range(0, n2) :
        sum1 = sum1 + v[i]
        
    return (sum1/n2)


income0_aver = aver(ds2.income.tolist())
income0_aver
#check
ds2.income.mean()

##########

def ssquare (v) :
    
    ssum = 0

    for i in range(0, n2) :
        ssum = ssum + v[i]*v[i]
  
    return (ssum)


def stdv (av, ss) :
    sd = np.sqrt((ss - n2*av*av) / (n2-1))
    return (sd)


av = aver(ds2.income.tolist())
ss = ssquare(ds2.income.tolist())

income0_std = stdv(av, ss)
ds2.income.std()



a1 = [4, 3, 2, 1]
a2 = pd.Series([5, 5, 6, 6])

a1
a2
ds2.income  #series type
ds2['income'] #series type
ds2.income.tolist() #list type

print(type(ds2.income))
print(type(ds2.income.tolist()))
print(a2.index)
print(a2.values)
type(ds2.income.tolist())
```